In [2]:
from langchain_community.embeddings import GPT4AllEmbeddings
from pinecone import Pinecone as PineconeClient
from langchain.chat_models import init_chat_model
from langchain_pinecone import PineconeVectorStore
import os
from dotenv import load_dotenv
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import START, StateGraph
from langchain_core.prompts import PromptTemplate
from langchain_deepseek import ChatDeepSeek
import re, json

In [3]:
# Load env
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = PineconeClient(
      api_key=PINECONE_API_KEY,
  )
index_name = "langchainretrieval"
index = pc.Index(index_name)

In [5]:
embedding = GPT4AllEmbeddings()
vector_store = PineconeVectorStore(embedding=embedding, index_name=index_name)

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-pro-exp-02-05",
    api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0.0
)

### 1. Define State, Nodes, Control Flow in LangGraph

In [28]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    meta: dict

In [ ]:
def retrieve(state: State):
    prompt_template = PromptTemplate(
            input_variable=["question"],
            template="""Tìm các thông tin location, rating, hotel_name (nếu có) trong câu: {question}. 
            Nếu không có thông tin thì loại bỏ biến đó khỏi kết quả.
            Trả về JSON chỉ chứa các key có giá trị, theo format sau:
            {{"location": "string", "rating": int, "hotel_name": "string"}}"""
    )
    filter = llm.invoke(prompt_template.invoke({"question": state['question']}))
    meta = re.sub(r'```json\n|```', '', filter.content).strip()
    meta = json.loads(meta)
    retrieved_docs = vector_store.similarity_search(state['question'], 
                                                    k=30,
                                                    filter=meta)
    return {"context": retrieved_docs, "meta": meta}

def generate(state: State):
    meta = state["meta"] if "meta" in state else {}
    if not state['context']:
        return {"answer": f"Xin lỗi, hệ thống không tìm thấy khách sạn nào ở {meta.get('location', '')} với rating {meta.get('rating', '')}. Vui lòng chọn lại rating khác."}
    docs_content = "\n\n".join(f"{doc.page_content}\nmetadata: {doc.metadata}" for doc in state['context'])
    prompt_generation = PromptTemplate(
        input_variables=['context'],
        template="""- Tóm tắt thông tin của tất cả khách sạn trong {context}, sử dụng dữ liệu từ metadata để bổ sung thông tin nhưng không in trực tiếp metadata.
        Không cần in ra ID của khách sạn. Nếu có các khách sạn trùng ID trong context thì có thể nhóm vào 1 khách sạn (vì cùng 1 khách sạn) in ra với các thông tin
        'Địa chỉ, Mô tả, Đánh giá (rating), URL_khách sạn.'
        Ngoài ra, nếu nhiều hơn 1 khách sạn thì câu đầu tiên nên ghi là 'Dưới đây là thông tin một số khách sạn ở location với rating' (map location và rating tương ứng ở metadata).
        Còn hỏi khách sạn cụ thể thì câu đầu nên ghi là 'Dưới đây là thông tin khách sạn hotel_name' (map hotel_name ở metadata)."""
    )
    messages = prompt_generation.invoke({"context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [30]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

### 2. RAG and Generation

In [63]:
results = graph.invoke({"question": "Thông tin khách sạn Homestay Nhân Linh"})

In [64]:
print(results['answer'])

Dưới đây là thông tin khách sạn Homestay Nhân Linh:

*   **Địa chỉ:** 78 Nguyên Tử Lực phuong 8 dalat, Đà Lạt, Việt Nam
*   **Mô tả:** Nằm cách Vườn hoa Đà Lạt 1 km, Nhân Linh có chỗ nghỉ với sảnh khách chung, khu vườn và lễ tân 24 giờ để tạo thuận tiện cho du khách. Nhà nghỉ B&B này cung cấp miễn phí cả WiFi lẫn chỗ đỗ xe riêng. Một số căn tại đây được bố trí khu vực ghế ngồi và/hoặc ban công. Du khách nghỉ tại Nhân Linh có thể thư giãn trên sân hiên. Hồ Xuân Hương và Công viên Yersin đều nằm trong bán kính 2,7 km từ chỗ nghỉ. Sân bay gần nhất là sân bay Liên Khương, cách Nhân Linh 31 km.
*   **Đánh giá:** 5.0
*   **URL\_khách sạn:** [https://www.booking.com/hotel/vn/nhan-linh.vi.html?label=gen173nr-1FCAEoggI46AdIKlgEaPQBiAEBmAEquAEXyAEM2AEB6AEB-AECiAIBqAIDuALJm9ybBsACAdICJDE3MzdlYWUwLTU1ZDktNGQzNi1hMGZlLTU5MGRkZDc0ODY5MtgCBeACAQ&sid=842627633388b4367a2fb42d0ca3ab7f&aid=304142&ucfs=1&arphpl=1&dest\_id=-3712045&dest\_type=city&group\_adults=2&req\_adults=2&no\_rooms=1&group\_children=

In [65]:
print(results['context'])

[Document(id='7218be4b-b432-4349-b241-1bd9889c9fa0', metadata={'hotel_id': 2696.0, 'hotel_name': 'Homestay Nhân Linh', 'location': 'Đà Lạt', 'rating': 5.0}, page_content='Tên khách sạn: Homestay Nhân Linh\nMô tả: Nằm cách Vườn hoa Đà Lạt 1 km, Nhân Linh có chỗ nghỉ với sảnh khách chung, khu vườn và lễ tân 24 giờ để tạo thuận tiện cho du khách. Nhà nghỉ B&B này cung cấp miễn phí cả WiFi lẫn chỗ đỗ xe riêng. Một số căn tại đây được bố trí khu vực ghế ngồi và/hoặc ban công. Du khách nghỉ tại Nhân Linh có thể thư giãn trên sân hiên. Hồ Xuân Hương và Công viên Yersin đều nằm trong bán kính 2,7 km từ chỗ nghỉ. Sân bay gần nhất là sân bay Liên Khương, cách Nhân Linh 31 km.\nĐịa chỉ: 78 Nguyên Tử Lực phuong 8 dalat, Đà Lạt, Việt Nam'), Document(id='e4842ace-1dcc-4089-aaa0-1d12f9049aae', metadata={'hotel_id': 2696.0, 'hotel_name': 'Homestay Nhân Linh', 'location': 'Đà Lạt', 'rating': 5.0}, page_content='Địa chỉ: 78 Nguyên Tử Lực phuong 8 dalat, Đà Lạt, Việt Nam\nURL khách sạn: https://www.boo